# 🦜🔗 LangChain React Agent Demo

In diesem Part schauen wir uns das Thema Agenten an. Dabei nutzten wir das neueste Langchain Format für Agenten, LangGraph. Mit LangGraph können Agenten als Graphen modelliert werden.

Um die Komplexität gering zu halten, verwenden wir in diesen Beispielen prebuilt Agent- und Tool-Executors, die von Langchain bereit gestellt werden.

### Pakete installieren:

In [ ]:
%pip install -r requirements.txt

### Umgebungsvariablen laden:

In [ ]:
from dotenv import load_dotenv

load_dotenv()

### Tools:

Mit Tools erlangt der Agent zusätzliche Fähigkeiten und Wissen, die über die vorhandenen Fähigkeiten des eingesetzten Sprachmodells hinausgehen. Tools werden dabei vom Agenten immer eigenständig eingesetzt.

Wie vorab erwähnt, nutzen wir einen Prebuilt Tool-Executor um die Komplexität gering zu halten.

Als Tool setzen wir hier eine Websuche über DuckduckGo ein. Dadurch erlangt der Agent die Fähigkeit Informationen aus dem Internet zu recherchieren.

In [ ]:
from langgraph.prebuilt import ToolExecutor
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

tools = [DuckDuckGoSearchRun()]
tools_executor = ToolExecutor(tools)

### Agent

In diesem Schritt erstellen wir den Agenten selbst. 

Hierfür benötigen wir ein LLM mit dem der Agent ausgeführt werden soll. **Wichtig** ist hierbei, dass das Sprachmodell auf die Ausführung von Agenten-Logik trainiert sein muss. Wir verwenden in diesem Beispiel die Modelle von OpenAI, da diese optimal für die Ausführung von Agenten-Logik geeignet sind.

Damit LangGraph die Agenten-Logik ausführen kann, benötigen wir einen Agent-Executor. Auch hier nutzen wir einen Prebuilt Agent-Executor, in diesem Beispiel einen der OpenAI Functions ausführen kann.

Mit .stream führen wir den Graphen aus und geben uns die einzelen Steps des Agentenlaufs aus.

In [ ]:
from langgraph.prebuilt import chat_agent_executor
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from helpers import llm

# app = chat_agent_executor.create_tool_calling_executor(llm, tools_executor)
model = llm()
app = chat_agent_executor.create_function_calling_executor(model, tools)

inputs = {"messages": [HumanMessage(content="Wann wurde Dieter Bohlen geboren? Wann seine Frau? Was ist die Altersdifferenz?")]}

for s in app.stream(inputs):
    print(list(s.values()))
    print("-----")

In diesem Beispiel führen wir den zuvor definierten Agenten nochmals aus und streamen diesmal allerdings nur die finale Antwort des Agentenlauf raus. Dabei wird die Antwort Token für Token ausgegeben.

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import chat_agent_executor
from helpers import graph_agent_llm_output_streamer_events, llm

inputs = {"messages": [HumanMessage(content="Wann wurde Dieter Bohlen geboren? Wann seine Frau? Was ist die Altersdifferenz?")]}

model = llm()

app = chat_agent_executor.create_function_calling_executor(model, tools)

await graph_agent_llm_output_streamer_events(app, inputs)